In [ ]:
"""This code compares the resistivity values obtained from the contact-free probe and the results from a Van der
Pauw measurement in the PPMS. This code uses paramters obtained in STO_Standard.ipynb. For reference, please see 
'Probe-2.pdf' and my thesis that I emailed previously. """

import matplotlib.ticker as ticker
import numpy as np
from numpy import linalg as LA
import cmath
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.family"] = "Times New Roman"
from scipy.interpolate import approximate_taylor_polynomial
from scipy.interpolate import interp1d
import pandas as pd
import matplotlib.colors as mcolors 

height = 0.528 #mm
f=671111 #frequency in Hz
a = 2*255/237 #x dimension in mm
b=2*214/237 #y dimension in mm

res = 49 #Minimum resolution in each direction
d = np.linspace(0.001,20,1000) #d stores values of the parameter X, where X=a*b/delta^2 (delta is the skin depth and sqrt(a*b) is the characteristic length scale.)
chis = np.zeros(1000,dtype=np.complex_) #Stores the effective magnetic susceptibility for each d value of a rectangular sample (a by b)
for u in range(len(d)):
    delta = math.sqrt(a*b/d[u]) #skin depth
    k=(1-1j)/delta #k value
    minn = np.maximum(a,b)
    div = np.maximum(5*int(minn/delta),res) #This analysis is only useful if there are many more 
    #divisions per unit length than the inverse of the skin depth (Otherwise, valuable information is missed).
    #This is the reason for the scaling resolution at small skin depths
    #print("Number of divisions: ", div**2)
    dx = a/(div+1) #stepsize: x direction
    dy = b/(div+1) #stepsize: y direction
    H0 = 1 #External field magnitude

    """This analysis solves the differential equation del^2(H)+k^2*H=0 over a rectangular sample.
    A second order approximation for H is used, which is appropriate given the presence of the Laplacian."""

    Xmat = np.zeros([div,div],dtype=np.complex_) #A
    Ymat = np.zeros([div,div],dtype=np.complex_) #B
    for i in range(div):
        for j in range(div):
            if (i ==j):
                Xmat[i][j]= -2/dx**2+k**2
                Ymat[i][j]= -2/dy**2
            elif (abs(i-j) == 1):
                Xmat[i][j]= 1/dx**2
                Ymat[i][j]= 1/dy**2


    C = np.zeros([div,div],dtype=np.complex_) #boundary conditions (The component of H parallel to
    #the surface of the conductor is continuous provided there are no free surface currents.)
    space =0
    for i in range(div):
        for j in range(div):
            if ((i==0) or (i == (div-1))):
                C[i][j] -= H0/dx**2 
            if ((j==0) or (j == (div-1))):
                C[i][j] -= H0/dy**2
    #"""


    #"""
    #The steps below were taken from http://aero-comlab.stanford.edu/Papers/jameson_007.pdf 
    #X is the solution to the matrix equation AX + XB = C

    mu, U = LA.eig(Xmat)
    lam, V = LA.eig(Ymat)

    Chat = LA.inv(U)@C@V

    Xhat = np.zeros([div,div],dtype=np.complex_)
    for i in range(div):
        for j in range(div):
            Xhat[i][j] = Chat[i][j]/(mu[i]+lam[j]) 

    X = U@Xhat@LA.inv(V)
    for i in range(div):
        for j in range(div):
            if X[i][j].real > 1:
                print("Error at position ",i,", ",j)
    NewX= np.zeros([div+2,div+2],dtype=np.complex_) 
    for i in range(div+2):
        for j in range(div+2):
            if ((i ==0) or (i==(div+1)) or (j ==0) or (j==(div+1))):
                NewX[i][j] =1 
            else:
                NewX[i][j]=X[i-1][j-1]
    summ = 0
    for i in range(div+1):
        for j in range(div+1):
            summ += (NewX[i][j]+NewX[i+1][j]+NewX[i][j+1]+NewX[i+1][j+1])*dx*dy/4
    chis[u]=summ/(H0*a*b)-1
#fig,bx = plt.subplots(1, 1, sharex = True, figsize=(10, 8))
#bx.plot(d,chis.real)
#bx.plot(d,chis.imag)

im = np.zeros(len(chis))
#bm=2.206 #See Mag-STO.ipynb
T = 0.2188 #Phase offset in radians, determined from a superconductor


#for i in range(len(chis)):
    #im[i] = bm*(-chis[i]*np.exp(T*1j)).imag*height/1.054

#F=interp1d(d,im) #TEST

probe_dat = pd.read_excel('/Users/jackzwettler/Documents/Ge_contact.xlsx') #Retrieve data (contact-free)
contact_dat = pd.read_excel('/Users/jackzwettler/Documents/Ge_res.xlsx') #Retrieve data (Four-wire from PPMS)

Xdat = probe_dat.to_numpy() 
Cdat = contact_dat.to_numpy()

low = 3088
high = 9260

clow = 4
chigh = 1810

RT = np.zeros(high-low+1) #Temperature data for this iteration only
RY = np.zeros(high-low+1) #phase " "

CT = np.zeros(chigh-clow+1) 
CP = np.zeros(chigh-clow+1)
    
for i in range(high-2,low-3,-1):
    RT[i-low+2]=Xdat[i][0] #T (K)
    RY[i-low+2]=Xdat[i][5]/1000 #Y (mV)
for i in range(chigh-2,clow-3,-1):
    CT[i-clow+2]=Cdat[i][2] #T (K)
    CP[i-clow+2]=Cdat[i][8] #Resistivity (uohmcm)
M=2.206326530612245 #Determined from STO_Standard.ipynb
#T = np.linspace(0.2,0.27,50) #theta
T=0.2188
#C = np.linspace(33,36,50) #mapping R to resistivity

Xm = np.zeros(len(RT))
im = np.zeros(len(chis))


V_S = (1.054*3*298/453*3*305/453) #Volume of the STO sample used in STO_Standard.ipynb in mm^3

for i in range(len(chis)):
    im[i] = M*(-chis[i]*np.exp(T*1j)).imag*(height*a*b)/V_S
F=interp1d(im,d)
Rho = np.zeros(len(RT))
color = np.zeros(len(RT))
colors=['red','green','blue']
for g in range(len(RT)):
    if RY[g]<min(im):
        Xm[g]=min(d) 
        color[g]=1
    elif RY[g]>max(im):
        Xm[g]=max(d) 
        color[g]=2
    else:
        Xm[g] = F(RY[g])
        color[g]=3
    Rho[g] = 40*np.pi**2*a*b*f/(Xm[g])/1000000 
fig,bx = plt.subplots(1, 1,figsize=(6, 4))
cmap, norm = mcolors.from_levels_and_colors([0, 1.5, 2.5, 3.5], ['red', 'green', 'blue']) 
bx.scatter(RT,Rho,s=1,c=color,cmap=cmap,norm=norm)
bx.scatter(CT,CP,s=1,color='orange')
bx.tick_params(axis='x', labelsize=15)
bx.tick_params(axis='y', labelsize=15)
bx.set_xlabel('Temperature (K)',fontsize = 15)
bx.set_ylabel('Resistivity ',fontsize = 15)
bx.legend(['Contact-free probe','Van der Pauw method'],fontsize = 15)
bx.grid()
bx.set_ylim([0,1000])